In [ ]:
# getting dependencies
import cohere
import cohere.classify as co_classify
import dotenv
import os
import numpy as np
import pandas as pd 

: 

In [ ]:
# happiness vector
day_decoding = ['happy', 'sad']

: 

In [ ]:
# setting up client from env file
dotenv.load_dotenv()
co_client = cohere.Client(f'{os.getenv("COHERE_KEY")}')

: 

In [ ]:
speech_text = "My laptop missing in class today"

response = co_client.classify(
    inputs = [f"{speech_text}"],
    examples = [
        co_classify.Example('My friend made me smile today', 'happy'),
        co_classify.Example('My family took me out today', 'happy'),
        co_classify.Example('Today my cats gave me a lick', 'happy'), 
        co_classify.Example('I went to the park, but it was rainy and stormy', 'sad'),
        co_classify.Example('At school, I won an award', 'happy'),
        co_classify.Example('Today, some bullying happened at my school. I feel bad for that student', 'sad'),
        co_classify.Example('I worked hard today, but it was all worth it', 'happy'),
        co_classify.Example('I worked too hard today, work was really rough', 'sad'),
        co_classify.Example('I was not able to win an award today, but my friend won something', 'happy'),
        co_classify.Example("A crime happened at my neighbor's home", 'sad'),
        co_classify.Example('I saw a cat on the street, I wish I could have brought it home', 'sad'),
        co_classify.Example("I met one of my idols today!", 'happy')
    ]
)

: 

In [ ]:
# vector encoding of happy and sad
response_labels = response.classifications[0].labels
happiness_encoding = ([response_labels['happy'].confidence, response_labels['sad'].confidence])

: 

In [ ]:
# determining the state of your day
print("speech:", speech_text)
print('rating:', day_decoding[np.argmax(happiness_encoding)])

print('/n', response_labels)

: 

In [ ]:
prompt = '''"At the park, there was a murderer at night and I almost was not able to escape with my life and as a result, I am traumatised"
In summary: "The park is dangerous at night"

"At the carnival, there was a variety of food to pick from and I was never happier"
In summary:"Lot's of food makes me happy"

"Powershell gave my computer a virus and now I have lost all functionality with it"
In summary:"'''
print(prompt)

n_generations = 5

prediction = co_client.generate(
    model='large',
    prompt=prompt,
    return_likelihoods = 'GENERATION',
    stop_sequences=['"'],
    max_tokens=50,
    temperature=0.7,
    num_generations=n_generations,
    k=0,
    p=0.75)


# Get list of generations
gens = []
likelihoods = []
for gen in prediction.generations:
    gens.append(gen.text)

    sum_likelihood = 0
    for t in gen.token_likelihoods:
        sum_likelihood += t.likelihood
    # Get sum of likelihoods
    likelihoods.append(sum_likelihood)

pd.options.display.max_colwidth = 200
# Create a dataframe for the generated sentences and their likelihood scores
df = pd.DataFrame({'generation':gens, 'likelihood': likelihoods})
# Drop duplicates
df = df.drop_duplicates(subset=['generation'])
# Sort by highest sum likelihood
df = df.sort_values('likelihood', ascending=False, ignore_index=True)
print('Candidate summaries for the sentence: \n"Killer whales have a diverse diet, although individual populations often specialize in particular types of prey."')
df

: 